# Week 3 Assignment - Encounter Plotting

Your task is to:

1.  Load example aircraft tracks from a file and plot them.
2.  Propose a statistical model for capturing features from the data.

Turn in your final Julia notebook file (.ipynb) on Coursework for part (1) and a document describing your proposed statistical model for part (2) to coursework.

### Aircraft Tracks

![Encounter](figures/encounter_plot.png)

An airspace encounter consists of two components - the initial conditions and the transitions over time.
One file is provided for each.

The initial conditions file, [`initial.txt`](http://web.stanford.edu/class/aa120q/data/initial_small.txt), contains a table with the following columns:

| Variable | Type | Description |
| -------- |:----:| ----------- |
| **id**       | Int  | trace id    |
| **A**      | Int  | index corresponding to the airspace class (i.e., B, C, D, or other) |
| **C1**    | Int   | category of aircraft 1 |
| **C2**    | Int   | category of aircraft 2 |

The transitions file, [`transition.txt`](http://web.stanford.edu/class/aa120q/data/transition_small.txt), contains a table with the following columns:

| Variable | Type | Description |
| -------- |:----:| ----------- |
| **id**                  | Int   | trace id, same as in `initial.txt`    |
| **t**                   | Int   | time in 1s intervals from 0 to 50    |
| **n1**                  | Int   | airplane 1 northing [ft/s] | **n2**                  | Int   | airplane 2 northing [ft/s]   |
| **e1**                  | Int   | airplane 1 easting [ft/s]  | **e2**                  | Int   | airplane 2 easting [ft/s]    |
| **h1**                  | Int   | airplane 1 altitude [ft]    | **h2**                  | Int   | airplane 2 altitude [ft]    |
| **v1**                  | Int   | airplane 1 horizontal speed [ft/s]    | **v1**                  | Int   | airplane 2 horizontal speed [ft/s]    |
| **ψ1**                  | Int   | airplane 1 heading [deg]    | **ψ2**                  | Int   | airplane 2 heading [deg]    |

Write a program to load these trajectories and plot them.

1.  Your algorithm must be implemented from scratch in Julia.
2.  Although you may discuss your algorithm with others, you must not share code.

Directions

* Implement `pull_encounter` to load a specific `Encounter` from initial and transition
* Modify `plot_encounter` to indicate the location of minimim separation

In [1]:
# Load initial.txt as a DataFrame
using DataFrames
initial = readtable(Pkg.dir("AA120Q", "data", "initial_small.txt"))

,id,A,C1,C2
1,1,4,2,2
2,2,4,2,2
3,3,2,2,2


In [2]:
# Load traces.txt as a DataFrame
traces = readtable(Pkg.dir("AA120Q", "data", "traces_small.txt"))
traces[1:3,:]

,id,t,n1,e1,h1,v1,ψ1,n2,e2,h2,v2,ψ2
1,1,0,-8663.356343901081,334.6189855606664,8273.006246536497,280.548,-13.715783000000005,2172.6419947992413,-4228.932261893066,1685.3174490867889,156.281,200.15960299999992
2,1,1,-8470.635911654446,381.65540917758483,8074.628853288174,280.548,-12.986118000000005,2025.9353555932234,-4175.072137739151,1685.3174490867889,154.82377,203.0110329999999
3,1,2,-8277.33211172748,426.23377933392,7876.25146003985,280.548,-12.256453000000006,1883.4309754667102,-4114.550229294595,1685.3174490867889,153.36654,205.8624629999999


`AircraftState` - a type which describes the aircraft state

In [3]:
immutable AircraftState
    n::Float64  # northing   [ft]
    e::Float64  # easting    [ft]
    h::Float64  # altitude   [ft]
    ψ::Float64  # heading    [deg] (angle CCW from north)
    v::Float64  # speed      [ft/s] (total speed in 3D)
    hd::Float64 # climb rate [ft/s] (vertical speed component)
end

`Encounter` - a type which describes an encounter

In [4]:
type Encounter
    A::Int  # airspace class
    C1::Int # category of aircraft 1
    C2::Int # category of aircraft 2
    Δt::Float64 # constant timestep between frames [sec]
    trace1::Vector{AircraftState} # aircraft states for AC1, ordered in time
    trace2::Vector{AircraftState} # aircraft states for AC1, ordered in time

    function Encounter(
        A::Int,
        C1::Int,
        C2::Int,
        Δt::Float64,
        trace1::Vector{AircraftState},
        trace2::Vector{AircraftState},
        )

        retval = new()
        retval.A = A
        retval.C1 = C1
        retval.C2 = C2
        retval.Δt = Δt
        retval.trace1 = trace1
        retval.trace2 = trace2
        retval
    end
end

In [22]:
function pull_encounter(initial::DataFrame, traces::DataFrame, id::Int)

    # This function will prepare an "Encounter" data structure for plotting (see below),
    # by initializing the aircraft traces (trace1, trace2) into arrays of 51 elements (use the
    # Array() function in Julia to initialize the aircraft traces). 
    
    # Use a for loop to populate the aircraft states of the trace arrays, from the initial and
    # transitional data tables provided (initial.txt, traces.txt).
    # You must calculate "Δt" from the time difference in the trace data.
    
    ################
    # YOUR CODE HERE
    ################
    
    Encounter(initial[id, :A], initial[id, :C1], initial[id, :C2],
              Δt, trace1, trace2)
    
end

pull_encounter (generic function with 1 method)

In [ ]:
using Plots
plotlyjs() # use the PlotlyJS backend

function plot_encounter(enc::Encounter)

    trace1, trace2 = enc.trace1, enc.trace2

    palette=[colorant"0x52E3F6", colorant"0x79ABFF"]
    t_arr = (collect(1:length(trace1)).-1) .* enc.Δt

    e1_arr = map(s->s.e,trace1)
    n1_arr = map(s->s.n,trace1)
    e2_arr = map(s->s.e,trace2)
    n2_arr = map(s->s.n,trace2)

    min_e1, max_e1 = extrema(e1_arr)
    min_n1, max_n1 = extrema(n1_arr)
    min_e2, max_e2 = extrema(e2_arr)
    min_n2, max_n2 = extrema(n2_arr)

    max_e = max(max_e1, max_e2)
    max_n = max(max_n1, max_n2)
    min_e = min(min_e1, min_e2)
    min_n = min(min_n1, min_n2)

    w = max(abs(max_e), abs(min_e), abs(max_n), abs(min_n)) + 500

    p1 = plot(Vector{Float64}[e1_arr, e2_arr],
              Vector{Float64}[n1_arr, n2_arr],
              xlabel="East (ft)", ylabel="North (ft)", palette=palette, linewidth=4, xlims=(-w,w), ylims=(-w,w))
    
    # initial positions
    scatter!(p1, Vector{Float64}[Float64[trace1[1].e], Float64[trace2[1].e]],
                 Vector{Float64}[Float64[trace1[1].n], Float64[trace2[1].n]])
    
    ################
    # YOUR CODE HERE
    
    # compute the minimum 3D separation distance between aircraft
    # and the index at which it occurs
    minimum_distance = Inf
    index_at_min_dist = 0
    
    ################
    
    i = index_at_min_dist
    d = minimum_distance
    
    p2 = plot(Vector{Float64}[t_arr, t_arr],
              Vector{Float64}[map(s->s.h,trace1), map(s->s.h,trace2)],
              xlabel="Time (s)", ylabel="Altitude (ft)", leg=false, palette=palette, linewidth=4,
              annotations=(i,(trace1[i].h+trace2[i].h)/2, Plots.text(@sprintf("mid dist: %.0f ft", d))))
 
    plot!(p1, [trace1[i].e, trace2[i].e], [trace1[i].n, trace2[i].n], color=colorant"0xFF007F", linewidth=4)
    plot!(p2, [(i-1)*enc.Δt,(i-1)*enc.Δt], [trace1[i].h, trace2[i].h], color=colorant"0xFF007F", linewidth=4)

    plot(p1, p2, size=(950,400))
end

In [24]:
# enc = pull_encounter(initial, traces, 1)
# plot_encounter(enc)

In [25]:
# enc = pull_encounter(initial, traces, 2)
# plot_encounter(enc)

In [26]:
# enc = pull_encounter(initial, traces, 3)
# plot_encounter(enc)

### Propose an Encounter Model

Now that you are familiar with the Probabilistic Models discussed in class (and described in section 2.1 of DMU), you will propose an encounter model. Your model should be designed such that you can sample from it and produce initial positions for each aircraft. Your model should also support propagating the aircraft forward - ie, describe how the encounter changes over time from the initial setup.

You must explain the reasoning behind your decision, how it captures the initial and transition distributions, and how it can be learned from large initial and transition data tables.